In [4]:
from idlelib.iomenu import encoding
!pip install dask
!pip install pyarrow
!pip install numpy
!pip install pandas
!pip install polars


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import dask.dataframe as dd
import polars as pl
import numpy as np
import os
import pandas as pd

### Wygenerowanie duzego pliku

In [9]:
rows_per_chunk = 1_000_000
total_rows = 10_000_000  # ok. 10GB
filename = "bigfile_polars.csv"
categories = np.array(['A', 'B', 'C', 'D'])
np.random.seed(42)

# Usuń plik, jeśli istnieje
if os.path.exists(filename):
    os.remove(filename)

for i, start in enumerate(range(0, total_rows, rows_per_chunk)):
    end = min(start + rows_per_chunk, total_rows)
    n_rows = end - start
    data = {
        "value": np.random.randint(0, 10_000, size=n_rows),
        "category": np.random.choice(categories, size=n_rows)
    }
    df = pl.DataFrame(data)
    if i == 0:
        df.write_csv(filename)
    else:
        with open(filename, "a", encoding='utf-8') as f:
            df.write_csv(f, include_header=False)
    print(f"Zapisano wiersze {start} - {end}")

print("Zrobione!")

Zapisano wiersze 0 - 1000000
Zapisano wiersze 1000000 - 2000000
Zapisano wiersze 2000000 - 3000000
Zapisano wiersze 3000000 - 4000000
Zapisano wiersze 4000000 - 5000000
Zapisano wiersze 5000000 - 6000000
Zapisano wiersze 6000000 - 7000000
Zapisano wiersze 7000000 - 8000000
Zapisano wiersze 8000000 - 9000000
Zapisano wiersze 9000000 - 10000000
Zrobione!


## Sprawdzenie różnic wydajności

In [10]:
# Pandas
import time
import dask.dataframe as dd
import time
import polars as pl
import time

import pandas as pd
import time

filename = "bigfile_polars.csv"
#############
start = time.time()
df = pd.read_csv(filename)
mean = df["value"].mean()
print("Pandas mean:", mean)
print("Czas:", time.time() - start)

start = time.time()
result = df.groupby("category")["value"].sum()
print("Pandas groupby sum:", result.head())
print("Czas:", time.time() - start)

###############
# polars
start = time.time()
df = pl.read_csv(filename)
mean = df["value"].mean()
print("Polars mean:", mean)
print("Czas:", time.time() - start)

# Polars
start = time.time()
result = df.group_by("category").agg(pl.col("value").sum())
print("Polars groupby sum:", result.head())
print("Czas:", time.time() - start)

start = time.time()

# Lazy scan (nie ładuje całego pliku do RAM)
df = pl.scan_csv(filename)

# Lazy pipeline, wynik będzie policzony podczas collect()
mean = df.select(pl.col("value").mean()).collect()
print("Polars scan mean:", mean)

print("Czas:", time.time() - start)
# Polars
start = time.time()
result = (
    pl.scan_csv(filename)
    .group_by("category")
    .agg(pl.col("value").sum())
    .collect()  # <-- TO JEST KLUCZ!
)
# print("Polars groupby sum:", result.head())
print("Polars groupby sum:")
print("Czas:", time.time() - start)

####################
# dask
start = time.time()
df = dd.read_csv(filename)
mean = df["value"].mean().compute()
print("Dask mean:", mean)
print("Czas:", time.time() - start)


Pandas mean: 5001.6439352
Czas: 1.3945930004119873
Pandas groupby sum: category
A    12500570937
B    12511713704
C    12496685862
D    12507468849
Name: value, dtype: int64
Czas: 0.46700024604797363
Polars mean: 5001.6439352
Czas: 0.28908419609069824
Polars groupby sum: shape: (4, 2)
┌──────────┬─────────────┐
│ category ┆ value       │
│ ---      ┆ ---         │
│ str      ┆ i64         │
╞══════════╪═════════════╡
│ A        ┆ 12500570937 │
│ D        ┆ 12507468849 │
│ C        ┆ 12496685862 │
│ B        ┆ 12511713704 │
└──────────┴─────────────┘
Czas: 0.1730034351348877
Polars scan mean: shape: (1, 1)
┌─────────────┐
│ value       │
│ ---         │
│ f64         │
╞═════════════╡
│ 5001.643935 │
└─────────────┘
Czas: 0.17600178718566895
Polars groupby sum:
Czas: 1.6470134258270264
Dask mean: 5001.6439352
Czas: 2.452009439468384
